In [1]:
import numpy as np
from scipy.linalg import sqrtm
from keras.models import load_model
from keras.datasets.mnist import load_data
import matplotlib.pyplot as plt
# MacOS matplotlib kernel issue
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

Using TensorFlow backend.
/Users/tejas/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/tejas/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/tejas/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/tejas/anaconda3/lib/python3.7/site-packages/tenso

In [2]:
(train_images, train_labels), (_, _) = load_data()
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
train_images = (train_images - 127.5) / 127.5

In [3]:
N_SAMPLES = 10000
IMAGE_SHAPE = (28, 28)

In [4]:
generator_model = load_model('./DigitModels/GENmodel_150.h5')
gen_shape = tuple(generator_model.input.get_shape().as_list()[1:])
gen_noise = np.random.normal(loc=0, scale=1, size=(N_SAMPLES,)+gen_shape)

created_digits = generator_model.predict(gen_noise)





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [5]:
def reshape_image(image, image_shape):
    image = ((image + 1) * 127.5) / 255
    return image.reshape(image_shape).astype(np.float32)

In [6]:
def fréchet_inception_distance(image1, image2):
    mean1, stddev1 = np.mean(image1, axis=0), np.cov(image1, rowvar=False)
    mean2, stddev2 = np.mean(image2, axis=0), np.cov(image2, rowvar=False)
    sum_squared_diff_mean = np.sum((mean1 - mean2) ** 2)
    root_prod_cov = sqrtm(stddev1.dot(stddev2))
    
    if np.iscomplexobj(root_prod_cov):
        root_prod_cov = root_prod_cov.real
    
    fid = sum_squared_diff_mean + np.trace(stddev1 + stddev2 - 2. * root_prod_cov)
    return fid

In [7]:
def get_metrics(real_images, generated_images):
    n_samples = len(real_images)
    
    tot_fid = []
    
    for ix in range(n_samples):
        flat_real = reshape_image(real_images[ix], IMAGE_SHAPE)
        flat_generated = reshape_image(generated_images[ix], IMAGE_SHAPE)
        fid = fréchet_inception_distance(flat_real, flat_generated)
        if np.isnan(fid):
            fid = 0
        tot_fid.append(fid)
    return np.array(tot_fid)

In [8]:
fid = get_metrics(train_images[:N_SAMPLES], created_digits)

Failed to find a square root.
Failed to find a square root.


In [9]:
np.mean(fid)

1.5224564806525505

In [10]:
np.std(fid)

0.7293499902718441